<a href="https://colab.research.google.com/github/FedericoCalonge/OrganizacionDatosRepoMateria/blob/master/5-SPARK/Notebook/SPARK_UBA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1-Inicio SPARK y Lectura de datos.

Vamos a ver cómo leer datos de un archivo con Spark y como hacerlo mediante transformaciones y acciones. 


## Instalamos e importamos librerías

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 204.7MB 62kB/s 
     |████████████████████████████████| 204kB 40.4MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.0-py2.py3-none-any.whl size=205044182 sha256=ac13f6e8dc3e354723a91dbafcc6f1f05b90d14a5fd52dc35b481a06ee268ff5
  Stored in directory: /root/.cache/pip/wheels/57/27/4d/ddacf7143f8d5b76c45c61ee2e43d9f8492fc5a8e78ebd7d37
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 35 not upgraded.
Need to get 35.8 MB of archives.
After this 

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

## Autenticamos con Google Drive

In [ ]:
#Esto lo hacemos para poder bajar archivos de Google Drive.
#Tenemos que entrar al link y colocar el código de verificación.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
#Esto es por si queremos montar nuestra Carpeta de Google al directorio de Files que vemos a la izquierda en el ícono "Archivos/Files" 
#(para así poder guardar o cargar archivos desde ahí).
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!ls "/content/drive/My Drive"

'Backups Obligatorios Sincronizados'
'Colab Notebooks'
 ContinuacionDjangoCurso4.rar
'CSVs TP1-OrgaDeDatos'
'Documento sin título.gdoc'
'Grupo Tecnico IA'
'HTML - CSS - BOOTSTRAP - PYTHONDJANGO.rar'
'TL4 Ejemplos'
'TP1 - Organizacion de Datos - Undav_Team'


## Bajamos archivo con la colección de Shakespeare y lo colocamos en la carpeta especificada

In [ ]:
#ARREGLAR ESTO. Al parecer cuando hice arriba el moun drive me tiró este error. VERLO Y DESPUES VER COMO ACCEDER AL id '1ybtSQxrqVqbRrl_3FMzMYW03Flp4zM-j' EN EL NAVEGADOR.

#Acá bajamos el archivo s.txt que son textos de Sheckspeare que vamos a usar más adelante para algunos ejemplos.
downloaded = drive.CreateFile({'id':"1ybtSQxrqVqbRrl_3FMzMYW03Flp4zM-j"})   # Este es el ID del archivo que queremos tener acceso (que lo tiene el profe en su Drive).
downloaded.GetContentFile("/content/drive/My Drive/Colab Notebooks/UBA/5-SPARK/Notebook/Txt Shekspeare utilizado/s.txt")  #Este txt lo vemos a la izquierda yendo al ícono "Archivos/Files" y lo vemos en esa ruta que lo coloqué.

AttributeError: ignored

## Creamos el Spark Context

In [ ]:
#Tenemos que crear el "Spark Context" para poder trabajar con Spark. 

# 1ro creamos una sesión de Spark:
spark = SparkSession.builder.getOrCreate()

# Y luego de esta variable spark (nuestra sesión) obtenemos el contexto de Spark:
sc = spark.sparkContext

#Y esta variable, sc, es la que vamos a utilizar para comunicarnos con Spark.

In [ ]:
type(sc)

pyspark.context.SparkContext

## Lectura de datos en Spark

Vamos a ver cómo leer datos en Spark. 
Vamos a ver 3 formas de hacerlo.

### Forma 1 - Paralelizando una coleccion de python

In [ ]:
#La idea de esta manera es tener una colección de datos en Python y paralelizarlos en un RDD con Spark.

integersList = range(1,1001) ## Creamos una lista de entero del 1 al 1000 (no incluye al 1001).
len(integersList)

1000

In [ ]:
#Ahora para transformar esta variable de Python en un RDD lo que hacemos es llamar al método
#parallelize (que es de sc, osea del Spark Context). Si ponemos el puntero del mouse sobre la función parallelize
# nos tira la info de ese método. Nos dice que parallelize permite distribuir una colección de Python local
#en un RDD, y el 2do parametro (slice, en el que pusimos 8) es la cantidad de particiones en que queremos que divida 
#Spark ese RDD.

#De esta manera paralelizamos la coleccion utilizando 8 particiones o slices
#Esta operacion es una transformacion de datos en un RDD. Y dado que Spark usa lazy evaluation, no corren jobs de Spark hasta el momento.

integersListRDD = sc.parallelize(integersList, 8) #Entonces le pasamos la lista de enteros y le decimos que lo divida en 8 particiones. 
type(integersListRDD) #Entonces ahora en integersListRDD tenemos un RDD de Spark que podemos utilizarlo para luego procesarlo.

pyspark.rdd.PipelinedRDD

In [ ]:
integersListRDD.getNumPartitions() #Para comprobar efectivamente que el RDD está dividido en 8 particiones (que fue lo que le indicamos cuando lo creamos)

8

In [ ]:
integersListRDD.toDebugString() #Método que nos permite llamar a un RDD que tenga transformaciones 
#para ver dichas transformaciones que se aplican en el RDD. Como en nuestro caso es un RDD que recién creamos solo
#tenemos la operación de paralelización ('ParallelCollectionRDD'). 

b'(8) PythonRDD[1] at RDD at PythonRDD.scala:53 []\n |  ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:262 []'

In [ ]:
help(integersListRDD) #Para ver más métodos disponibles para utilizar en estructuras RDDs.

Help on PipelinedRDD in module pyspark.rdd object:

class PipelinedRDD(RDD)
 |  Pipelined maps:
 |  
 |  >>> rdd = sc.parallelize([1, 2, 3, 4])
 |  >>> rdd.map(lambda x: 2 * x).cache().map(lambda x: 2 * x).collect()
 |  [4, 8, 12, 16]
 |  >>> rdd.map(lambda x: 2 * x).map(lambda x: 2 * x).collect()
 |  [4, 8, 12, 16]
 |  
 |  Pipelined reduces:
 |  >>> from operator import add
 |  >>> rdd.map(lambda x: 2 * x).reduce(add)
 |  20
 |  >>> rdd.flatMap(lambda x: [x, x]).reduce(add)
 |  20
 |  
 |  Method resolution order:
 |      PipelinedRDD
 |      RDD
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, prev, func, preservesPartitioning=False, isFromBarrier=False)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  getNumPartitions(self)
 |      Returns the number of partitions in RDD
 |      
 |      >>> rdd = sc.parallelize([1, 2, 3, 4], 2)
 |      >>> rdd.getNumPartitions()
 |      2
 |  
 |  id(self)
 |      A unique ID for this RD

In [ ]:
integersListRDD.take(5) #Llamamos a una ACCIÓN de Spark (take) que lo que hace es obtener los
#primeros 5 registros del RDD. En nuestro caso los números del 1 al 5. 

[1, 2, 3, 4, 5]

In [ ]:
integersListRDD.count() #Ahora ejecutamos otra ACCIÓN (count) que te devuelve la cantidad de registros del RDD.
#Como recordamos, integersListRDD lo creamos a partir de las variables en Python.

1000

### Forma 2 - Leyendo archivo con textFile

In [ ]:
#De esta manera vamos a leer el archivo 's.txt' con la función 'textFile':
rdd = spark.sparkContext.textFile('/content/drive/My Drive/Colab Notebooks/UBA/5-SPARK/Notebook/Txt Shekspeare utilizado/s.txt') #Acá ponemos al archivo 's.txt' que descargamos y pusimos en esa ruta previamente.

In [ ]:
#Este textFile nos devuelve directamente un RDD. 
rdd #Chequeamos que es una variable 'MapPartitionsRDD'.

/content/drive/My Drive/Colab Notebooks/UBA/5-SPARK/Notebook/Txt Shekspeare utilizado/s.txt MapPartitionsRDD[5] at textFile at NativeMethodAccessorImpl.java:0

In [ ]:
rdd.count()  #Vemos la cantidad de registros que tiene. 

124614

In [ ]:
rdd.take(5) #Vemos los primeros 5 registros: osea las 1ras 5 lineas del txt.

['1609', '', 'THE SONNETS', '', 'by William Shakespeare']

### Forma 3 - Leyendo datos con el sqlContext

In [ ]:
#Le pasamos el sc (el sparkContext).
sqlContext = SQLContext(sc)

In [ ]:
#Cargamos la información de nuestro s.txt a un DataFrame (nivel de abstracción "superior" al RDD: teniendo así nombre de columnas y demás datos):
dataframe = sqlContext.read.text('/content/drive/My Drive/Colab Notebooks/UBA/5-SPARK/Notebook/Txt Shekspeare utilizado/s.txt')

#Como vimos en la parte teórica, este dataframe es una API de mayor nivel que el RDD, y nos permite ejecutar trabajos de Map-Reduce con
#funciones de más alto nivel que el API de RDD.

Arriba pusimos 'read.text' pero tambien hay otras opciones para leer archivos csv, json, archivos con conexiones de BDs JDBC, etc. (para saber que esto tambien se puede).

In [ ]:
dataframe

DataFrame[value: string]

In [ ]:
#Y lo que podemos hacer para pasar de DF a RDD es esto:
rddCsv = dataframe.rdd

In [ ]:
rddCsv  #Ahora acá tenemos un RDD que podemos usarlo para procesarlo con acciones y transformaciones.

MapPartitionsRDD[14] at javaToPython at NativeMethodAccessorImpl.java:0

In [ ]:
rddCsv.take(5)

[Row(value='1609'),
 Row(value=''),
 Row(value='THE SONNETS'),
 Row(value=''),
 Row(value='by William Shakespeare')]

# 2-Acciones.

Vamos a ver algunas acciones que tiene disponible el API RDD de Spark.

## Count

Para obtener la cantidad de registros del RDD.

In [ ]:
integersListRDD.count()  #integersListRDD era la lista de enteros del 1 al 1000.

1000

### Take

Obtiene los primeros n registros del RDD.

In [ ]:
integersListRDD.take(5)   #Queremos obtener en este caso los 1ros 5 registros del RDD.

[1, 2, 3, 4, 5]

## Collect

Obtiene TODOS los registros del RDD. Esto es un potencial problema, ya que si los datos no son acotados (si son demasiados) entonces se va a sobrecargar el driver cuando ejecutamos Collect. SOLO se debe ejecutar si de antemano conocemos que la cantidad de datos es acotada.

In [ ]:
integersListRDD.collect() #CUIDADO! 
                          #como sabemos que integersListRDD tiene solo 1000 registros lo utilizamos, no hay problema.

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

## First

Obtiene el primer registro del RDD.

In [ ]:
integersListRDD.first()

1

## TakeOrdered

Obtiene los primeros n registros en base a un ORDEN indicado (mediante una función de ordenamiento).

In [ ]:
integersListRDD.takeOrdered(5, key=lambda x: -x) 
  #1er parámetro (5): cantidad de registro que queremos obtener.
  #2do parámetro: es la key de ordenamiento. En este caso decimos que la key del ordenamiento es = a la función lambda x - x... osea que
  #estamos ordenando los números de forma descendente. 
  #De esta manera obtenemos los números más grandes (si quisieramos los más chicos simplemente pondríamos "lambda x: x")

[1000, 999, 998, 997, 996]

## TakeSample

Obtiene una muestra aleatoria de n registros con o sin reemplazo.

In [ ]:
integersListRDD.takeSample(False, 5)
  #1er parametro: variable booleana que me dice si la muestra es CON (true) o SIN (false) reemplazo (osea si
  #un registro puede ser elegido más de 1 vez).
  #2do parámetro: cantidad de registro que quermeos obtener.
  #3er paráemtros opcional: la semilla, seed. Es para el tema de aleatoriedad. Si queremos obtener los mismos registros ponemos la misma semilla, sino ponemos otra.

[125, 517, 994, 680, 172]

## Reduce

Obtiene un solo registro, combinando el resultado en base a una función dada (En base a una función Reduce que toma de a 2 valores).

Por ej. para SUMAR todos los nros del RDD utilizamos la acción Reduce y le pasamos un lambda que recibe 2 registros de nombre a y b... y lo que hacemos es sumar a y b. De esta manera lo que hacemos es SUMAR DE A PARES TODOS LOS NÚMEROS. Y como resultado obtenemos la suma de los mil números de nuestro RDD.

In [ ]:
integersListRDD.reduce(lambda a,b: a+b)

500500

Entonces, dependiendo de la función lambda en la función reduce que utilicemos, podemos obtener distinta información del RDD. Si por ejemplo ahora queremos **obtener el número más grande del RDD**... hacemos una función lambda que se quede con el número más grande entre a y b. 

Utilizar esta función para obtener el MIN o el MAX de un RDD cuando tenemos muchos datos **es mucho más eficiente que hacer un ordenamiento** previo (con takeordered) y luego obtener el mayor o menor de ese ordenamiento: esto último es muy costoso cuando tenemos muchos datos. 

In [ ]:
integersListRDD.reduce(lambda a,b: a if a > b else b) #Se queda con a si (if) es mayor a b y sino (else) se queda con b.

1000

## CountByKey

Cuenta ocurrencias de registros para cada clave.

En Spark **para que un registro sea considerado que tenga una clave debe ser una tupla de únicamente dos elementos. El primer elemento es la key y el segundo el valor.** A su vez, cada key y cada valor pueden estar compuestos por tuplas compuestas... entonces quizás la key es otra tupla de 2 elementos y el valor es una tupla de 5 elementos.

De esta forma, si tenemos un registro que es una tupla de 2 elementos podemos ejecutar transformaciones y acciones que trabajen con clave. Y si trabajamos con registros que NO tienen 2 elementos entonces NO podemos utilizar las transformaciones o acciones que trabajen con una key; y si necesitamos si o si utilizar una de estas deberíamos mapear primero los datos para que tenga una tupla de 2 elementos. 

Acá por ejemplo contamos cuántos números múltiplo de 2 hay y cuántos no:

In [ ]:
#1ro mapeamos cada uno de los registros (usando la transformación MAP). De esta manera x es
#el 1er elemento (clave) y le ponemos un 0 o un 1 indicando si es o no múltiplo de 2 (Si es par o impar).
#Y en el 2do elemento (Valor) simplemente le ponemos un 1 SIEMPRE. 
integersListRDD.map(lambda x: (x % 2, 1)).take(10)  #Tomamos los 1ros 10 elementos.

[(1, 1),
 (0, 1),
 (1, 1),
 (0, 1),
 (1, 1),
 (0, 1),
 (1, 1),
 (0, 1),
 (1, 1),
 (0, 1)]

In [ ]:
#Y ahora si usamos countByKey() para contar cuántos registros hay de cada clave... vemos que hay 500 elementos pares (cuando el módulo es 0, clave 0)
#y 500 números impares (clave=1). 
integersListRDD.map(lambda x: (x % 2, 1)).countByKey()

defaultdict(int, {0: 500, 1: 500})

# 3-Transformaciones.

### Map

Transforma cada registro en base a la función dada.

In [ ]:
#MAP recibe una función que indica cómo transformar cada uno de los registros del RDD.
#En este caso usamos la función de lamba x*2.. osea vamos a multiplicar x 2 cada uno de nuestros registros y luego obtenemos con take los 1ros 5.
integersListRDD.map(lambda x: x*2).take(5)

[2, 4, 6, 8, 10]

In [ ]:
#La función map NO es necesario que reciba 1 número y devuelva 1 número... acá abajo recibimos 1 número x pero
#devolvemos una tupla de 2 elementos (parecido a como hicimos 2 lineas de código arriba, solo que ahora le vamos a colocar el número (x) envés de un simple 1 en la posición del 'valor'. En la posición
#de la 'clave' seguirá un 0 si el número es múltiplo de 2 o un 1 si no lo es.
integersListRDD.map(lambda x: (x % 2, x)).take(5)  #Y tomamos los 1ros 5 números.

#Vemos que 1 es impar, 2 es par, 3 impar, 4 par, 5 impar y asi...
#De esta forma transformamos nuestros números en una tupla indicando si el número es par o no.
#Como es una tupla de 2 elementos entonces SPARK reconoce al primero (al 0 o al 1) como el elemento 'key' y
#al resto de los números (1,2,3,4,5,6...) como los 'value'. De esta manera me permitiría ejecutar en estas tuplas transformaciones o acciones que trabajen con clave.

[(1, 1), (0, 2), (1, 3), (0, 4), (1, 5)]

## Filter

Filtra registros en base a la función dada.

In [ ]:
#Esta función FILTER devuelve un TRUE o FALSE en base a si cumple o no la función. 
#Si queremos mantener ese registro devuelve TRUE y si queremos que lo descarte devuelve FALSE.
#Filtramos / descartamos los regisotrs quedandonos solo con los registros que son PARES.
integersListRDD.filter(lambda x: x % 2 == 0).take(5)

[2, 4, 6, 8, 10]

In [ ]:
integersListRDD.filter(lambda x: x % 2 == 0).count()

500

## FlatMap

Similar a Map, pero cada registro puede generar 0, 1 o más registros (NO hay limitaciones en la cantidad de registros que podemos dar como salida usando FlatMap).

De esta manera, con flatMap recibimos una función de transformación. En este caso en lambda x devolvemos 3 registros para cada registro original: Un registro que tiene 'x', otro que es 'x-1', y otro que es 'x+1'.

Así, para cada registro original generamos 3 nuevos registros.

In [ ]:
integersFlat = integersListRDD.flatMap(lambda x: [(x), (x-1), (x+1)])
integersFlat.count() #De esta manera, como originalmente teníamos 1000 registros, ahora tendremos 3000.

3000

In [ ]:
integersFlat.take(6)  #Así, el 1er registro que era 1... ahora tenemos 1 (Que es el mismo
#registro), 0 (registro-1) y 2 (registro+1). Y para el 2do registro que era 2 lo mismo (tenemos 2, 1 y 3):

[1, 0, 2, 2, 1, 3]

## ReduceByKey

Combina los registros que tengan una misma clave en base a una función de Reduce. Como dijimos, una función de Reduce recibe 2 registros a la vez y los combina con una función. Acá recordemos que la función de Reduce debe ser **conmutativa** y **asociativa** para que el resultado tenga sentido (ya que va trabajando de forma distribuida de a pares). 

Del RDD salida del flatMap que hicimos anteriormente (en el que teniamos 3000 registros), ahora contamos cuantos registros hay para cada nro:

In [ ]:
#integersFlat son los 3000 enteros que obtuvimos del flatMap anterior.
#Del map devolvemos el número x y un 1 --> '(x,1)'. Y luego aplicamos un reduceByKey y le pasamos
#la función de Reduce donde hacemos simplemente a+b. Acá hay que tener en cuenta que estos a y b NO son todos
#los registros, sino que es únicamente la parte del valor del registro (osea el 2do elemento de la tupla)... osea en este caso
#va a estar recibiendo el 1 --> por el '(x,1)' anterior. 
#Entonces, para todos los registros que tengan la misma clave va a estar recibiendo ese 2do 1 y entonces
#va a estar sumando 1+1+1+1+1 y así... (en el 'a+b') en base a todos los registros que hayan para esa clave.
integersFlat.map(lambda x: (x, 1)).reduceByKey(lambda a,b: a+b).count()   #Y tambien ejecutamos la acción de count() para contar.

#Tenemos 1000 registros que cumplen esto... los 1000 registros originales + el 1 inicial que le restamos y el 1 final que le sumamos 
#(El 0 y el 1001). Vamos a ver este 0 y 10001 para comprobar abajo...

1002

In [ ]:
#Primero vemos que el 0 que dijimos previamente:
integersFlat.map(lambda x: (x, 1)).reduceByKey(lambda a,b: a+b).take(10)
#Acá vemos que tenemos solo un 0 (el que dijimos), tres 8s, tres 16s, etc. Vamos a tener 3 de toodos los demás números (menos del 1001 que vemos abajo si está al final o no)...

[(0, 1),
 (8, 3),
 (16, 3),
 (24, 3),
 (32, 3),
 (40, 3),
 (48, 3),
 (56, 3),
 (64, 3),
 (72, 3)]

In [ ]:
#Y ahora vemos el final para comprobar que está el 1001 que dijimos previamente:
#De esta manera usamos un reduce que recibe un lambda a,b y devolvemos solo el mayor. 
integersFlat.map(lambda x: (x, 1)).reduceByKey(lambda a,b: a+b).reduce(lambda a,b: a if a > b else b)

(1001, 1)

## GroupByKey

Agrupa los registros para cada clave. 


Es similar a reduceByKey pero con groupByKey se obtiene todos los registros para cada clave. Enves de aplicar una función de Reduce para cada uno de los registros de la misma clave, lo que va a hacer GroupByKey es AGRUPAR los registros para cada clave y nos lo va a devolver en cada uno de los registros... osea ahora vamos a tener en cada registro la clave como 1er elemento y como 2do elemento a cada uno de los valores que tenia esa clave en los registros originales (osea si teniamos 3 registros para la clave 1, luego de realizar el GroupByKey vamos a tener un registro que es clave 1 y como valor cada uno de estos 3 registros... y esto lo va a hacer para cada una de las claves distintas que aparezcan en el RDD). 

**Esto se utiliza en casos muy particulares, ya que es una operación MUY costosa**. La mayoría de las veces envés de utilizar GroupByKey podemos usar ReduceByKey. GroupByKey se usa únicamente cuando por alguna razón necesitamos hacer algo con los registros ORIGINALES y no alcanza con hacer una función de agregación a los registros. Además de ser costosa nos puede traer problemas... ya que quizás para una sola clave hayan cientos de miles de registros que tengan esa clave... entonces va a pasar que cada registro va a ser de un tamaño muy grande.

**Hay que usarlo con cuidado y SOLO si es MUY necesario. Hay que pensar previamente si nuestro problema no se puede resolver con un ReduceByKey. Si se desea realizar una agregación, usar reduceByKey. Usar groupByKey para hacer una agregación esta MAL.**

Antes habíamos calculado cuántos eran múltiplos de 2 y cuántos no. Ahora queremos saber CUALES son los nros múltiplos de 2 y cuales no:

In [ ]:
#Y con esto obtenemos:
  #un 0 como clave y como valor una lista tooooodos los valores pares, 
  #o un 1 como clave y como valor una lista con tooooodos los valores impares. 
integersListRDD.map(lambda x: (x % 2, x)).groupByKey().map(lambda x: (x[0], list(x[1]))).collect()
#El 2do map es totalmente innecesario, es solo para que se pueda ver... si no la ejecutamos la parte de los registros agrupados se van a ver como iterables de RDD y no se
#va a haber el contenido; entonces transformando cada uno de estos registros en un 'list' podemos ver el contenido.

[(0,
  [2,
   4,
   6,
   8,
   10,
   12,
   14,
   16,
   18,
   20,
   22,
   24,
   26,
   28,
   30,
   32,
   34,
   36,
   38,
   40,
   42,
   44,
   46,
   48,
   50,
   52,
   54,
   56,
   58,
   60,
   62,
   64,
   66,
   68,
   70,
   72,
   74,
   76,
   78,
   80,
   82,
   84,
   86,
   88,
   90,
   92,
   94,
   96,
   98,
   100,
   102,
   104,
   106,
   108,
   110,
   112,
   114,
   116,
   118,
   120,
   122,
   124,
   126,
   128,
   130,
   132,
   134,
   136,
   138,
   140,
   142,
   144,
   146,
   148,
   150,
   152,
   154,
   156,
   158,
   160,
   162,
   164,
   166,
   168,
   170,
   172,
   174,
   176,
   178,
   180,
   182,
   184,
   186,
   188,
   190,
   192,
   194,
   196,
   198,
   200,
   202,
   204,
   206,
   208,
   210,
   212,
   214,
   216,
   218,
   220,
   222,
   224,
   226,
   228,
   230,
   232,
   234,
   236,
   238,
   240,
   242,
   244,
   246,
   248,
   250,
   252,
   254,
   256,
   258,
   260,
   262,


In [ ]:
#Ejecutamos el collect sin el 2do MAP solo para ver...:
integersListRDD.map(lambda x: (x % 2, x)).groupByKey().collect()

#Vemos que para la clave 0 y 1 tenemos 2 iterables pero no podemos ver el contenido, por eso antes lo transformamos a list.

[(0, <pyspark.resultiterable.ResultIterable at 0x7ff51409e470>),
 (1, <pyspark.resultiterable.ResultIterable at 0x7ff51409e390>)]

## Distinct

Elimina registros duplicados (todo el registro debe coincidir.. osea que TODO el registro debe ser igual para que la función Distinct lo pueda eliminar).

Se lo aplicamos a la lista integersFlat (el RDD trás aplicar flatMap que vimos anteriormente, donde teníamos 3000 números/registros) para obtener así los registros únicos:

In [ ]:
integersFlat.distinct().count()
#Son 1002 registros ya que teniamos los números del 1 al 1000, el 0 y el 1001. 

1002

# 4- Más ejemplos de transformaciones y acciones con los textos de Shakespeare (con uso de wordsCount).

## Leo de a líneas

In [ ]:
lines = spark.sparkContext.textFile('/content/drive/My Drive/Colab Notebooks/UBA/5-SPARK/Notebook/Txt Shekspeare utilizado/s.txt')

## Cantidad de líneas totales

In [ ]:
lines.count()

124614

## Primeras 10 líneas

In [ ]:
lines.take(10)

['1609',
 '',
 'THE SONNETS',
 '',
 'by William Shakespeare',
 '',
 '',
 '',
 '                     1',
 '  From fairest creatures we desire increase,']

## Obtengo las palabras de todas las líneas (flatMap)

In [ ]:
words = lines.flatMap(lambda x: x.split())

In [ ]:
words.take(10)

['1609',
 'THE',
 'SONNETS',
 'by',
 'William',
 'Shakespeare',
 '1',
 'From',
 'fairest',
 'creatures']

In [ ]:
words.count()

902892

## Contando palabras (reduceByKey)

In [ ]:
wordsCount = words.map(lambda x: (x.lower(),1))

In [ ]:
wordsCount.take(10)

[('1609', 1),
 ('the', 1),
 ('sonnets', 1),
 ('by', 1),
 ('william', 1),
 ('shakespeare', 1),
 ('1', 1),
 ('from', 1),
 ('fairest', 1),
 ('creatures', 1)]

In [ ]:
wordsCounted = wordsCount.reduceByKey(lambda x,y: x+y)

In [ ]:
wordsCounted.take(10)

[('shakespeare', 258),
 ('1', 13),
 ('fairest', 39),
 ('creatures', 27),
 ('we', 3210),
 ('increase,', 9),
 ('thereby', 21),
 ("beauty's", 30),
 ('rose', 44),
 ('never', 959)]

In [ ]:
wordsCounted.takeOrdered(10, lambda x: -x[1])

[('the', 27681),
 ('and', 26066),
 ('i', 19540),
 ('to', 18737),
 ('of', 18084),
 ('a', 14424),
 ('my', 12456),
 ('in', 10721),
 ('you', 10666),
 ('that', 10489)]

### Mal uso de groupByKey

In [ ]:
wordsCount.groupByKey().takeOrdered(10, lambda x: -1 * len(x[1]))

[('the', <pyspark.resultiterable.ResultIterable at 0x7ff51409eac8>),
 ('and', <pyspark.resultiterable.ResultIterable at 0x7ff5140f70b8>),
 ('i', <pyspark.resultiterable.ResultIterable at 0x7ff51409e4a8>),
 ('to', <pyspark.resultiterable.ResultIterable at 0x7ff5140f7390>),
 ('of', <pyspark.resultiterable.ResultIterable at 0x7ff51409e710>),
 ('a', <pyspark.resultiterable.ResultIterable at 0x7ff5140f71d0>),
 ('my', <pyspark.resultiterable.ResultIterable at 0x7ff5140f7978>),
 ('in', <pyspark.resultiterable.ResultIterable at 0x7ff51409e128>),
 ('you', <pyspark.resultiterable.ResultIterable at 0x7ff5140f79e8>),
 ('that', <pyspark.resultiterable.ResultIterable at 0x7ff5140f7ac8>)]

In [ ]:
wordsCount.groupByKey().map(lambda a: (a[0], list(a[1]))).take(5)

[('shakespeare',
  [1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,

In [ ]:
wordsCount.groupByKey().filter(lambda x: len(x[1]) < 5).map(lambda a: (a[0], list(a[1]))).take(5)

[('riper', [1, 1, 1]),
 ('memory:', [1]),
 ("feed'st", [1, 1, 1]),
 ("light's", [1]),
 ('fuel,', [1])]

## Palabra más larga (reduce)

In [ ]:
words.reduce(lambda a, b: a if (len(a) > len(b)) else b)

'http://www.ibiblio.org/gutenberg/etext06'

## Palabras que empiezan con a (filter)

In [ ]:
wordsA = words.filter(lambda word: word.startswith('a'))

In [ ]:
wordsA.count()

63676

In [ ]:
wordsA.take(10)

['as', 'a', 'abundance', 'art', 'and', 'a', 'asked,', 'all', 'all', 'an']

## Palabras únicas que empiezan con a (distinct)

In [ ]:
wordsA.distinct().count()

2688

## Cantidad de palabras por frecuencia de repetición ordenados (sortByKey)

In [ ]:
wordsCounted.take(5)

[('shakespeare', 258),
 ('1', 13),
 ('fairest', 39),
 ('creatures', 27),
 ('we', 3210)]

In [ ]:
wordsFreq = wordsCounted.map(lambda x: (x[1],1))

In [ ]:
wordsFreq.take(10)

[(258, 1),
 (13, 1),
 (39, 1),
 (27, 1),
 (3210, 1),
 (9, 1),
 (21, 1),
 (30, 1),
 (44, 1),
 (959, 1)]

In [ ]:
wordsFreq.reduceByKey(lambda a,b: a+b).sortByKey().take(10)

[(1, 31072),
 (2, 8493),
 (3, 4342),
 (4, 2659),
 (5, 1822),
 (6, 1338),
 (7, 1053),
 (8, 779),
 (9, 700),
 (10, 549)]

In [ ]:
wordsFreq.reduceByKey(lambda a,b: a+b).takeOrdered(10, lambda x: -x[1])

[(1, 31072),
 (2, 8493),
 (3, 4342),
 (4, 2659),
 (5, 1822),
 (6, 1338),
 (7, 1053),
 (8, 779),
 (9, 700),
 (10, 549)]

# 5-Transformaciones entre dos RDD (con uso de Joins).


## Union

Obtiene la unión entre dos RDD.

In [ ]:
integersList2 = range(501,1501)
len(integersList2)

1000

In [ ]:
integersList2RDD = sc.parallelize(integersList2)

In [ ]:
integersList2RDD.count()

1000

In [ ]:
integersListRDD.count()

1000

In [ ]:
union = integersListRDD.union(integersList2RDD)

In [ ]:
union.take(5)

[1, 2, 3, 4, 5]

In [ ]:
union.count()

2000

## Intersection

Intersección entre dos RDD.

In [ ]:
intersection = integersListRDD.intersection(integersList2RDD)

In [ ]:
intersection.count()

500

In [ ]:
intersection.take(10)

[510, 520, 530, 540, 550, 560, 570, 580, 590, 600]

In [ ]:
intersection.collect()

[510,
 520,
 530,
 540,
 550,
 560,
 570,
 580,
 590,
 600,
 610,
 620,
 630,
 640,
 650,
 660,
 670,
 680,
 690,
 700,
 710,
 720,
 730,
 740,
 750,
 760,
 770,
 780,
 790,
 800,
 810,
 820,
 830,
 840,
 850,
 860,
 870,
 880,
 890,
 900,
 910,
 920,
 930,
 940,
 950,
 960,
 970,
 980,
 990,
 1000,
 501,
 511,
 521,
 531,
 541,
 551,
 561,
 571,
 581,
 591,
 601,
 611,
 621,
 631,
 641,
 651,
 661,
 671,
 681,
 691,
 701,
 711,
 721,
 731,
 741,
 751,
 761,
 771,
 781,
 791,
 801,
 811,
 821,
 831,
 841,
 851,
 861,
 871,
 881,
 891,
 901,
 911,
 921,
 931,
 941,
 951,
 961,
 971,
 981,
 991,
 502,
 512,
 522,
 532,
 542,
 552,
 562,
 572,
 582,
 592,
 602,
 612,
 622,
 632,
 642,
 652,
 662,
 672,
 682,
 692,
 702,
 712,
 722,
 732,
 742,
 752,
 762,
 772,
 782,
 792,
 802,
 812,
 822,
 832,
 842,
 852,
 862,
 872,
 882,
 892,
 902,
 912,
 922,
 932,
 942,
 952,
 962,
 972,
 982,
 992,
 503,
 513,
 523,
 533,
 543,
 553,
 563,
 573,
 583,
 593,
 603,
 613,
 623,
 633,
 643,
 653,
 66

## Subtract

Elimina del primer RDD los registros que aparezcan en el segundo.

In [ ]:
subtract = integersListRDD.subtract(integersList2RDD)

In [ ]:
subtract.count()

500

In [ ]:
subtract.collect()

[10,
 20,
 30,
 40,
 50,
 60,
 70,
 80,
 90,
 100,
 110,
 120,
 130,
 140,
 150,
 160,
 170,
 180,
 190,
 200,
 210,
 220,
 230,
 240,
 250,
 260,
 270,
 280,
 290,
 300,
 310,
 320,
 330,
 340,
 350,
 360,
 370,
 380,
 390,
 400,
 410,
 420,
 430,
 440,
 450,
 460,
 470,
 480,
 490,
 500,
 1,
 11,
 21,
 31,
 41,
 51,
 61,
 71,
 81,
 91,
 101,
 111,
 121,
 131,
 141,
 151,
 161,
 171,
 181,
 191,
 201,
 211,
 221,
 231,
 241,
 251,
 261,
 271,
 281,
 291,
 301,
 311,
 321,
 331,
 341,
 351,
 361,
 371,
 381,
 391,
 401,
 411,
 421,
 431,
 441,
 451,
 461,
 471,
 481,
 491,
 2,
 12,
 22,
 32,
 42,
 52,
 62,
 72,
 82,
 92,
 102,
 112,
 122,
 132,
 142,
 152,
 162,
 172,
 182,
 192,
 202,
 212,
 222,
 232,
 242,
 252,
 262,
 272,
 282,
 292,
 302,
 312,
 322,
 332,
 342,
 352,
 362,
 372,
 382,
 392,
 402,
 412,
 422,
 432,
 442,
 452,
 462,
 472,
 482,
 492,
 3,
 13,
 23,
 33,
 43,
 53,
 63,
 73,
 83,
 93,
 103,
 113,
 123,
 133,
 143,
 153,
 163,
 173,
 183,
 193,
 203,
 213,
 223,
 233

## Joins

Con los joins se combinan dos RDD en base a las claves de los registros. Junta cada registro del primer RDD con cada registro del segundo RDD que tengan la misma clave. No agrupa, sino que es de a pares de registro.

In [ ]:
data_alumnos = [
  (1,'Damian'),
  (2,'Luis'),
  (3,'Martin'),
  (4,'Natalia'),
  (5,'Joaquin')
]

alumnos = sc.parallelize(data_alumnos)

In [ ]:
alumnos.collect()

[(1, 'Damian'), (2, 'Luis'), (3, 'Martin'), (4, 'Natalia'), (5, 'Joaquin')]

In [ ]:
data_materias_aprobadas = [
  (1, 'Algebra'),
  (2, 'Análisis Matemático'),
  (200, 'Algebra'),
  (2, 'Física')
]

materias_aprobadas = sc.parallelize(data_materias_aprobadas)

In [ ]:
materias_aprobadas.collect()

[(1, 'Algebra'), (2, 'Análisis Matemático'), (200, 'Algebra'), (2, 'Física')]

### Inner Join (Join)

Cuando se llama para sets de datos del tipo (K,V) y (K,W) devuelve un set de datos del tipo (K, (V,W)) con todos los pares de elementos para cada key. (especificamente los que hay en comun por esa clave en ambos sets de datos)

In [ ]:
alumnos.join(materias_aprobadas).collect()

[(1, ('Damian', 'Algebra')),
 (2, ('Luis', 'Análisis Matemático')),
 (2, ('Luis', 'Física'))]

### Left Outer Join

Cuando se llama para sets de datos del tipo (K,V) y (K,W) devuelve un set de datos del tipo (K, (V,W)) asegurandonos que todos los datos del set de datos izquierdo estaran en el resultado del join.

In [ ]:
alumnos.leftOuterJoin(materias_aprobadas).collect()

[(4, ('Natalia', None)),
 (1, ('Damian', 'Algebra')),
 (5, ('Joaquin', None)),
 (2, ('Luis', 'Análisis Matemático')),
 (2, ('Luis', 'Física')),
 (3, ('Martin', None))]

### Right Outer Join

Cuando se llama para sets de datos del tipo (K,V) y (K,W) devuelve un set de datos del tipo (K, (V,W)) asegurandonos que todos los datos del set de datos derecho estaran en el resultado del join.

In [ ]:
alumnos.rightOuterJoin(materias_aprobadas).collect()

[(200, (None, 'Algebra')),
 (1, ('Damian', 'Algebra')),
 (2, ('Luis', 'Análisis Matemático')),
 (2, ('Luis', 'Física'))]

### Outer/Full Join


Cuando se llama para sets de datos del tipo (K,V) y (K,W) devuelve un set de datos del tipo (K, (V,W)) asegurandonos que todos los datos de ambos set de datos estaran aunque no haya match de keys.

In [ ]:
alumnos.fullOuterJoin(materias_aprobadas).collect()

[(4, ('Natalia', None)),
 (200, (None, 'Algebra')),
 (1, ('Damian', 'Algebra')),
 (5, ('Joaquin', None)),
 (2, ('Luis', 'Análisis Matemático')),
 (2, ('Luis', 'Física')),
 (3, ('Martin', None))]

### Broadcast Join (map-side join)

#### Variable Broadcast

Una variable Broadcast nos permite mantener una variable solo lectura cacheada en cada una de las maquinas del cluster en vez de enviar esa informacion con cada una de las tareas que se envian al cluster.

Esto es particularmente util cuando cuando tareas a partir de multiples etapas (stages) necesitan la misma información o cuando cachear información de forma deserializada es importante.

Tener en cuenta que esto **es posible** cuando uno de los data sets o conjunto de datos **es lo suficientemente pequeño para ser broadcasteado a todos los nodos/workers del cluster**.

In [ ]:
# Vamos a suponer que tenemos un RDD de productos por sus IDs identificando ventas de los mismos
prodsList = [1,11,1,4,5,11,2,3,4,5,6,4,5,4,3,2,1,11,2,3,4,5,6,4,3,2,1,1]
prods = sc.parallelize(prodsList,3)

In [ ]:
# Un hash con los productos y sus nombres
productNames = {1:'papas',
                2:'cebollas',
                3:'tomates',
                4:'zanahorias',
                5:'batatas',
                6:'peras',
                7:'cilantro',
                8:'apio',
                9:'morrones',
                10:'manzanas',
                11:'naranjas'}

# Hacemos un broadcast de la variable
bproductNames = sc.broadcast(productNames)

In [ ]:
# Buscamos los productos que se vendieron más de 4 veces
popularProds = prods.map(lambda x:(x,1))\
    .reduceByKey(lambda x,y:x+y)\
    .filter(lambda x:x[1]>=4)
popularProds.collect()

[(3, 4), (1, 5), (4, 6), (5, 4), (2, 4)]

El join se realiza de forma implicita usando un map y dentro del mismo accediendo a la informacion de la variable a la que se realizo el broadcast via .value

In [ ]:
popularProds = popularProds.map(
    lambda x:(bproductNames.value[x[0]],x[0],x[1]))
popularProds.collect()

[('tomates', 3, 4),
 ('papas', 1, 5),
 ('zanahorias', 4, 6),
 ('batatas', 5, 4),
 ('cebollas', 2, 4)]

#### Ventajas

Cuando un valor es "broadcasteado" al cluster, este es copiado a los nodos/workers **sólo una vez** (en vez de múltiples veces si la información fuera a enviarse en cada task). De esta forma se resuelve la consulta más rapidamente.

# 6-Transformaciones sobre las particiones

In [ ]:
rdd = sc.parallelize(range(1,11))
rdd.getNumPartitions()

2

In [ ]:
sc.defaultParallelism

2

In [ ]:
rdd.collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

## Glom

Junta los registros de cada partición en una lista.

In [ ]:
rdd.glom().collect()

[[1, 2, 3, 4, 5], [6, 7, 8, 9, 10]]

## MapPartitions

Devuelve un nuevo RDD aplicando una función a cada partición del RDD.

In [ ]:
def f(iterator): yield __builtin__.sum(iterator)
rdd.mapPartitions(f).collect()

[15, 40]

## Repartition

Reshuffle los datos en el RDD de forma aleatoria para crear más o menos particiones y balancearlas. 

Hace un shuffle de todo los datos por la red.

In [ ]:
rdd = sc.parallelize(range(1,11), 4)
rdd.getNumPartitions()

4

In [ ]:
rdd.glom().collect()

[[1, 2], [3, 4, 5], [6, 7], [8, 9, 10]]

In [ ]:
rdd2 = rdd.repartition(2)

In [ ]:
rdd2.getNumPartitions()

2

In [ ]:
rdd2.glom().collect()

[[1, 2, 6, 7, 8, 9, 10], [3, 4, 5]]

Spark no hace shuffle de registros individuales sino de a bloques con un mínimo (no es un problema cuando se manejan grandes cantidades de datos)

## Coalesce

Decrementa la cantidad de particiones del RDD.

No hace shuffle por defecto, solo pasa datos de una partición a otra.

No quedan balanceadas.

In [ ]:
rddCoalesce = rdd.coalesce(2)

In [ ]:
rddCoalesce.glom().collect()

[[1, 2, 3, 4, 5], [6, 7, 8, 9, 10]]

## RepartitionAndSortWithinPartitions

Reparticiona un RDD de acuerdo a un particionador y ordena los registros en base a su clave.

Los registros deben tener clave.

Es más eficiente que hacer un repartition y luego un sort dentro de cada partición ya que realiza el sort en el mismo paso de shuffle.

In [ ]:
rdd.map(lambda x: (x, x)).collect()

[(1, 1),
 (2, 2),
 (3, 3),
 (4, 4),
 (5, 5),
 (6, 6),
 (7, 7),
 (8, 8),
 (9, 9),
 (10, 10)]

In [ ]:
rdd.map(lambda x: (x, x)).glom().collect()

[[(1, 1), (2, 2)],
 [(3, 3), (4, 4), (5, 5)],
 [(6, 6), (7, 7)],
 [(8, 8), (9, 9), (10, 10)]]

### Ascending

In [ ]:
rdd.map(lambda x: (x, x)).repartitionAndSortWithinPartitions(2).glom().collect()

[[(2, 2), (4, 4), (6, 6), (8, 8), (10, 10)],
 [(1, 1), (3, 3), (5, 5), (7, 7), (9, 9)]]

In [ ]:
rdd.map(lambda x: (x % 3, x)).repartitionAndSortWithinPartitions(2).glom().collect()

[[(0, 3), (0, 6), (0, 9), (2, 2), (2, 5), (2, 8)],
 [(1, 1), (1, 4), (1, 7), (1, 10)]]

In [ ]:
rdd.map(lambda x: (x % 3, x)).repartitionAndSortWithinPartitions(2, ascending=False).glom().collect()

[[(2, 2), (2, 5), (2, 8), (0, 3), (0, 6), (0, 9)],
 [(1, 1), (1, 4), (1, 7), (1, 10)]]

### PartitionFunc

In [ ]:
rdd.map(lambda x: (x * 2, x)).repartitionAndSortWithinPartitions(2).glom().collect()

[[(2, 1),
  (4, 2),
  (6, 3),
  (8, 4),
  (10, 5),
  (12, 6),
  (14, 7),
  (16, 8),
  (18, 9),
  (20, 10)],
 []]

In [ ]:
rdd.map(lambda x: (x * 2, x)).repartitionAndSortWithinPartitions(2, partitionFunc=lambda x: (x % 3)).glom().collect()

[[(2, 1), (6, 3), (8, 4), (12, 6), (14, 7), (18, 9), (20, 10)],
 [(4, 2), (10, 5), (16, 8)]]

# 7-Persistiendo RDD (Persistencia y CACHE).

## Cache

Cachea un RDD intermedio que va a ser utilizado varias veces de modo de evitar tener que ejecutar todas las transformaciones cada vez.

In [ ]:
rdd = sc.parallelize(range(1,100000))

In [ ]:
rddCached = rdd.map(lambda x: x*10).cache()

In [ ]:
rddCached.count()

99999

In [ ]:
rddCached.take(10)

[10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

## SaveAsTextFile

Guarda un RDD a disco en un archivo de texto.

In [ ]:
rdd.saveAsTextFile('numbers.txt')  #Podemos indicar la ruta acá tambien.

In [ ]:
rddN = sc.textFile('numbers.txt')

In [ ]:
rddN.collect()

Py4JJavaError: ignored

## SaveAsPickleFile

Guarda un RDD a disco en un archivo con los datos serializados.

In [ ]:
rdd.saveAsPickleFile('numbers2.file')

In [ ]:
rddN2 = sc.pickleFile('numbers2.file')

In [ ]:
rddN2.take(10)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]